<a href="https://colab.research.google.com/github/atis06/tensorflow_bilstm_pooling/blob/master/Thesis_net_poc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!wget https://github.com/amirziai/cse6242-project/raw/master/resources/datasets/bbc-text.csv

--2020-02-11 20:11:15--  https://github.com/amirziai/cse6242-project/raw/master/resources/datasets/bbc-text.csv
Resolving github.com (github.com)... 192.30.253.112
Connecting to github.com (github.com)|192.30.253.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/amirziai/cse6242-project/master/resources/datasets/bbc-text.csv [following]
--2020-02-11 20:11:16--  https://raw.githubusercontent.com/amirziai/cse6242-project/master/resources/datasets/bbc-text.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5057493 (4.8M) [text/plain]
Saving to: ‘bbc-text.csv’

bbc-text.csv        100%[===================>]   4.82M  --.-KB/s    in 0.06s   

2020-02-11 20:11:16 (74.5 MB/s) - ‘bbc-text.csv’ saved [5057

In [37]:


import csv
import tensorflow as tf
import numpy as np
import sklearn
import nltk
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords

nltk.download('stopwords')
STOPWORDS = set(stopwords.words('english'))

print(tf.__version__)

vocab_size = 5000
embedding_dim = 64
max_length = 200
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<UNK>'
training_portion = .8

articles = []
labels = []

with open("bbc-text.csv", 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    for row in reader:
        labels.append(row[0])
        article = row[1]
        for word in STOPWORDS:
            token = ' ' + word + ' '
            article = article.replace(token, ' ')
            article = article.replace(' ', ' ')
        articles.append(article)
print(len(labels))
print(len(articles))

train_size = int(len(articles) * training_portion)

train_articles = articles[0: train_size]
train_labels = labels[0: train_size]

validation_articles = articles[train_size:]
validation_labels = labels[train_size:]

print(train_size)
print(len(train_articles))
print(len(train_labels))
print(len(validation_articles))
print(len(validation_labels))

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_articles)
word_index = tokenizer.word_index
dict(list(word_index.items())[0:10])

train_sequences = tokenizer.texts_to_sequences(train_articles)
print(train_sequences[10])

train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
print(len(train_sequences[0]))
print(len(train_padded[0]))

print(len(train_sequences[1]))
print(len(train_padded[1]))

print(len(train_sequences[10]))
print(len(train_padded[10]))

print(train_padded[10])

validation_sequences = tokenizer.texts_to_sequences(validation_articles)
validation_padded = pad_sequences(validation_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

print(len(validation_sequences))
print(validation_padded.shape)

label_tokenizer = Tokenizer()
label_tokenizer.fit_on_texts(labels)

training_label_seq = np.array(label_tokenizer.texts_to_sequences(train_labels))
validation_label_seq = np.array(label_tokenizer.texts_to_sequences(validation_labels))
'''print(training_label_seq[0])
print(training_label_seq[1])
print(training_label_seq[2])
print(training_label_seq.shape)

print(validation_label_seq[0])
print(validation_label_seq[1])
print(validation_label_seq[2])
print(validation_label_seq.shape)'''

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])


def decode_article(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])


print(decode_article(train_padded[10]))
print('---')
print(train_articles[10])

print(train_padded.shape)
print(train_labels[0])



def next_batch(batch_size):
    idx = np.arange(0, len(train_padded))
    np.random.shuffle(idx)
    idx = idx[:batch_size]
    data_shuffle = [train_padded[i] for i in idx]
    labels_shuffle = [training_label_seq[i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)


X_batch, y_batch = next_batch(1)



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
1.15.0
2225
2225
1780
1780
1780
445
445
[2431, 1, 225, 4994, 22, 641, 587, 225, 4994, 1, 1, 1663, 1, 1, 2431, 22, 565, 1, 1, 140, 278, 1, 140, 278, 796, 822, 662, 2307, 1, 1144, 1695, 1, 1722, 4995, 1, 1, 1, 1, 1, 4737, 1, 1, 122, 4513, 1, 2, 2873, 1506, 352, 4738, 1, 52, 341, 1, 352, 2172, 3962, 41, 22, 3793, 1, 1, 1, 1, 543, 1, 1, 1, 835, 631, 2366, 347, 4739, 1, 365, 22, 1, 787, 2367, 1, 4301, 138, 10, 1, 3664, 682, 3531, 1, 22, 1, 414, 822, 662, 1, 90, 13, 633, 1, 225, 4994, 1, 600, 1, 1695, 1021, 1, 4996, 807, 1865, 117, 1, 1, 1, 2974, 22, 1, 99, 278, 1, 1607, 4997, 543, 492, 1, 1447, 4740, 778, 1320, 1, 1862, 10, 33, 641, 319, 1, 62, 478, 565, 301, 1507, 22, 479, 1, 1, 1666, 1, 797, 1, 3066, 1, 1365, 6, 1, 2431, 565, 22, 2971, 4734, 1, 1, 1, 1, 1, 850, 39, 1826, 675, 297, 26, 979, 1, 882, 22, 361, 22, 13, 301, 1507, 1343, 374, 20, 63, 883, 1096, 4302, 247]
426
20

In [58]:
######################################################################


import tensorflow as tf

from tensorflow.keras.layers import LSTMCell, Bidirectional

# Just one feature, the time series
num_inputs = 200
# Num of steps in each batch
num_time_steps = 1
# 100 neuron layer, play with this
num_hidden = 5
# Just one output, predicted time series
num_outputs = 1
# learning rate you can play with this
learning_rate = 0.000001
# how many iterations to go through (training steps), you can play with this
epochs = 800
# Size of the batch of data
batch_size = 10
n_classes = 5

'''with tf.Session() as sess:
    print(tf.one_hot(y_batch, n_classes,
                     axis=-1).eval())'''


def RNN(x, weights, biases, timesteps, num_hidden):
    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, timesteps, 1)

    # Define a rnn cell with tensorflow
    rnn_cell = tf.contrib.rnn.BasicRNNCell(num_hidden)

    # Get lstm cell output
    # If no initial_state is provided, dtype must be specified
    # If no initial cell state is provided, they will be initialized to zero
    states_series, current_state = tf.contrib.rnn.static_rnn(rnn_cell, x, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(states_series[-1], weights) + biases

def BiRNN(x, weights, biases, timesteps, num_hidden):
    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, num_input)

    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, num_input)
    x = tf.unstack(x, timesteps, 1)

    # Define lstm cells with tensorflow
    # Forward direction cell
    lstm_fw_cell = tf.contrib.rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)
    # Backward direction cell
    lstm_bw_cell = tf.contrib.rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)

    # Get BiRNN cell output
    outputs, _, _ = tf.contrib.rnn.static_bidirectional_rnn(lstm_fw_cell, lstm_bw_cell, x,
                                                 dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights) + biases

tf.compat.v1.reset_default_graph()
X = tf.placeholder(tf.float32, [None, num_time_steps, num_inputs])
y = tf.placeholder(tf.float32, [None, n_classes])

W = tf.Variable(tf.random_normal(shape=[2*num_hidden, n_classes]))
b = tf.Variable(tf.zeros([n_classes]))

'''cell = tf.contrib.rnn.OutputProjectionWrapper(
    tf.contrib.rnn.BasicLSTMCell(num_units=num_hidden, activation=tf.nn.relu),
    output_size=num_outputs)

print(tf.unstack(X, num_time_steps, 1))

outputs, state = tf.nn.dynamic_rnn(cell, tf.unstack(X, num_time_steps, 1), dtype=tf.float32)'''

# output_logits = tf.matmul(state, W) + b
output_logits = BiRNN(X, W, b, num_time_steps, num_hidden)
y_pred = tf.nn.softmax(output_logits)

cls_prediction = tf.argmax(output_logits, axis=1, name='predictions')


loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=output_logits))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
correct_prediction = tf.equal(tf.argmax(output_logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(epochs):

        X_batch, y_batch = next_batch(batch_size)
        # print(X_batch.shape)
        X_batch = X_batch.reshape((batch_size, num_time_steps, num_inputs))
        y_batch = tf.one_hot(y_batch-1, n_classes, axis=-1).eval().reshape(-1, n_classes)
        
        sess.run(optimizer, feed_dict={X: X_batch, y: y_batch})
        if epoch % 100 == 0:
            mse = loss.eval(feed_dict={X: X_batch, y: y_batch})
            print(epoch, "\tMSE:", mse)


    X_batch_valid = validation_padded.reshape((len(validation_padded), num_time_steps, num_inputs))
    y_batch_valid = tf.one_hot(validation_label_seq-1, n_classes, axis=-1).eval().reshape(-1, n_classes)

    feed_dict_valid = {X: X_batch_valid, y: y_batch_valid}

    loss_valid, acc_valid = sess.run([loss, accuracy], feed_dict=feed_dict_valid)
    print('---------------------------------------------------------')
    print("Epoch: {0}, validation loss: {1:.2f}, validation accuracy: {2:.01%}".
          format(epoch + 1, loss_valid, acc_valid))
    print('---------------------------------------------------------')


    y_pred = sess.run(cls_prediction, feed_dict={X: X_batch_valid[0].reshape(-1,1, num_inputs)})
    print(y_pred)
    index = tf.argmax(y_pred, axis=0).eval()
    print(index, y_batch_valid[0])


0 	MSE: 1.4226596
100 	MSE: 1.7893288
200 	MSE: 2.2262852
300 	MSE: 2.6442096
400 	MSE: 2.2937293
500 	MSE: 1.2811748
600 	MSE: 1.7587986
700 	MSE: 2.1783142
---------------------------------------------------------
Epoch: 800, validation loss: 1.90, validation accuracy: 22.5%
---------------------------------------------------------
[2]
0 [0. 0. 0. 0. 1.]


In [0]:
model = tf.keras.Sequential([
    # Add an Embedding layer expecting input vocab of size 5000, and output embedding dimension of size 64 we set at the top
    tf.keras.layers.Embedding(vocab_size, embedding_dim),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim)),
#    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    # use ReLU in place of tanh function since they are very good alternatives of each other.
    tf.keras.layers.Dense(embedding_dim, activation='relu'),
    # Add a Dense layer with 6 units and softmax activation.
    # When we have multiple outputs, softmax convert outputs layers into a probability distribution.
    tf.keras.layers.Dense(6, activation='softmax')
])
model.summary()

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
num_epochs = 10
history = model.fit(train_padded, training_label_seq, epochs=num_epochs, validation_data=(validation_padded, validation_label_seq), verbose=2)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          320000    
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               66048     
_________________________________________________________________
dense (Dense)                (None, 64)                8256      
_________________________________________________________________
dense_1 (Dense)  

KeyboardInterrupt: ignored